# Lab9 -- Security in Computer  Networks

## Introduction

This lab aims to introduce you to some of the security mechanisms used in computer networks. 
You will set up and analyze a firewall for packet filtering and a TLS for secure TCP connections.

For the final lab submission, provide a PDF report with answers to the questions labeled **REPORT** from labs 9 and 10. Questions marked with **Bonus** are optional.

## Lab Setup

[Figure 1](#figure_1) shows the setup for this lab. It is similar to the setup for [lab7](../lab7/figures/lab5.png), but the servers now have private IP addresses. 
Your job is to set up a firewall to allow secure access to the servers from external networks. 
Then, you will set up a website with TLS support.


<a id='figure_1'></a>

|<img src="figures/lab9.png" />|
|:--:| 
| *Figure 1: Lab 9 System Setup* |


- **Build the new container images** using the `create_docker_images.sh` script, before starting this lab.
You can use the `./create_docker_images.sh` command, when you are in the same directory as the file.
Otherwise, provide the entire path for the shell script as a command, e.g., `~/labs/lab9/create_docker_images.sh`

- **After building the images**, you can import the `lab9.gns3project` as a portable project in GNS3 to view the topology, access the containers, and start packet captures. You can still use `docker attach` in your VM in order to access the containers, as long as the topology is started in GNS3. In that case, be careful to detach and not exit the container.

- **The OSPF** protocol is already configured on the routers. Additionally, NAT is already configured on "router1" and "router2".

<div class="alert alert-block alert-info">
<b>Note:</b> To save configuration in the containers, run the save script provided in the home directory (<b>~</b>). Use <code style="color: black;">./save.sh</code> to run it. This will save any <b>nft</b> and <b>vtysh</b> configuration in the local copy of your GNS3 project.
</div>

In [ ]:
from test_lab9 import TestLab9
check_progress = TestLab9()

# Milestone 1 -- Firewall

In [lab7](../lab7/figures/lab7.png), for simplicity, we assigned public IP address to the webserver, which makes it available to any external network (i.e., the IP address is publicly routable across the entire internet), and anyone can access it in an unconstrained manner. 
But, this might not be ideal.

We should place our resources in an internal network ([Figure 1](#figure_1)) and should carefully scrutinize any access from an external network through a firewall.

<a id='sec:1.1'></a>

In this milestone, you will be using a Linux framework for packet filtering, NAT and firewall management, called [nftables](https://wiki.nftables.org/wiki-nftables/index.php/Quick_reference-nftables_in_10_minutes). This [video](https://www.youtube.com/watch?v=lHLPZlZLWgs) may help you understand the basic concepts and syntax.


## Task 1.1 -- [Port Forwarding](https://en.wikipedia.org/wiki/Port_forwarding)

The first step in building the firewall is to allow access to certain services residing on a private (protected) network.
We can use port forwarding to remap the public IP and port to that service's private IP address and port.

* In __"pclient2"__, verify that you can __not__ access the webserver (neither through the private IP of "webserver" nor through the public IP of "router0"):


   <div style="background: #f8f8f8; overflow:auto;width:auto;border:solid gray;border-width:.1em .1em .1em .8em;padding:.2em .6em;"><pre style="margin: 0; line-height: 125%;color: black;">wget http://10.20.30.3 --header "Host: www.ttm4200.com"<br><br>wget http://129.168.1.2 --header "Host: www.ttm4200.com"</pre></div>

* In __"router0"__, set up port forwarding to redirect any HTTP or HTTPS requests to the webserver. That means to set up a packet filter to inspect incoming packets: if a packet has a destination port of 80 (HTTP) or 443 (HTTPS), forwarded it to the webserver of the private IP 10.20.30.3:



   <div style="background: #f8f8f8; overflow:auto;width:auto;border:solid gray;border-width:.1em .1em .1em .8em;padding:.2em .6em;"><pre style="margin: 0; line-height: 125%;color: black;">sudo nft add table nat<br><br>sudo nft 'add chain nat postrouting { type nat hook postrouting priority 100 ; }'<br><br>sudo nft 'add chain nat prerouting { type nat hook prerouting priority -100; }'<br><br>sudo nft 'add rule nat prerouting ip daddr 129.168.1.2 tcp dport { 80, 443 } dnat to 10.20.30.3'<br><br>sudo nft add rule nat postrouting masquerade</pre></div>



<div class="alert alert-block alert-info">
<b>Note:</b> Before you continue, check <a href="https://jensd.be/1086/linux/forward-a-tcp-port-to-another-ip-or-port-using-nat-with-nftables" target="_blank">tutorial 1</a> and <a href="https://access.redhat.com/documentation/en-us/red_hat_enterprise_linux/7/html/security_guide/sec-configuring_port_forwarding_using_nftables" target="_blank">tutorial 2</a> to fully understand the previous commands. </div>


* In __"pclient2"__, verify that you can access the webserver by retrieving the content of its HTML page:


   <div style="background: #f8f8f8; overflow:auto;width:auto;border:solid gray;border-width:.1em .1em .1em .8em;padding:.2em .6em;"><pre style="margin: 0; line-height: 125%;color: black;">wget http://129.168.1.2 --header "Host: www.ttm4200.com"</pre></div>

In [ ]:
check_progress.test_1_1()

<a id='sec:1.2'></a>

## Task 1.2 -- Prevent Probing/Scanning a Network

One way to prevent probing/scanning your network is to block ICMP ping:


* In __"router0"__, drop all incoming ICMP echo requests, using nftables. You can use the following **pseudo instructions** and edit them to achieve this:

    <div style="background: #f8f8f8; overflow:auto;width:auto;border:solid gray;border-width:.1em .1em .1em .8em;padding:.2em .6em;"><pre style="margin: 0; line-height: 125%;color: black;">create a new table for ICMP filtering<br>add a chain with the input hook and type filter, with priority 0<br>add a rule to the input chain to drop ICMP echo-requests</pre></div>


<div class="alert alert-block alert-warning">
  <b>Important:</b> Pseudo instructions are here to help you solve the task. You can use <code style="color: black;"> nft list</code> to check your configuration. Similarly, you can use <code style="color: black;"> nft delete</code> to delete a specific rule entry, chain and table. Make sure <b>NOT</b> to flush nftables because it will remove all firewall rules, leaving the system unprotected.
</div>

<div class="alert alert-block alert-info">
<b>Note:</b> To solve <i>this and next task</i>, you need to understand nftables, check the links provided in the milestone's description. These are some suggested sources to read more:<br>
For the start, you can check how to create a <a href="https://wiki.archlinux.org/title/Nftables#Create_table" target="_blank">table</a>, <a href="https://wiki.archlinux.org/title/Nftables#Chains" target="_blank">chains</a> and how to add a <a href="https://wiki.archlinux.org/title/Nftables#Rules" target="_blank">rule</a> to the chain.<br>
In addition, you can take a look at the example rulesets <a href="https://wiki.nftables.org/wiki-nftables/index.php/Simple_ruleset_for_a_home_router" target="_blank">here</a>.
</div>

* Then, make sure you cannot ping __"router0"__.

* __Bonus:__ What is the downside of disabling ICMP? 

<div class="alert alert-block alert-info">
<b>Hint:</b> Check out this <a href="https://blog.paessler.com/disabling-icmp-and-snmp-wont-increase-security-but-will-impact-network-monitoring" target="_blank">link</a>. </div>

In [ ]:
check_progress.test_1_2()

<a id='sec:1.3'></a>

## 1.3 Filtering Inbound Traffic

A firewall on a home router allows you to initiate communications from internal devices but restricts any communication initiation from external networks. 
Your task in this section is to implement this firewall.


* Set up a firewall in __"router1"__ that:
   * Allow all connections initiated from internal devices.
   * Allows only established or [related](https://serverfault.com/questions/1037091/should-related-connections-always-be-allowed-in-iptables-ip6tables-nftables-et) connections from external networks
   * Otherwise, drop all incoming connections from external networks.
   * The following **pseudo instructions** may help you to solve this task:

    <div style="background: #f8f8f8; overflow:auto;width:auto;border:solid gray;border-width:.1em .1em .1em .8em;padding:.2em .6em;"><pre style="margin: 0; line-height: 125%;color: black;">create a new table for the firewall<br>add a chain with the forward hook and type filter, with priority 0. Drop packets by default<br>add a ct state rule to the forward chain to accept established and related packets<br>add an iifname rule to the forward chain to accept packets from ether0</pre></div>

* __Self-Check:__ Try to ping "pclient1" from "router3". Then, ping "router3" from "pclient1". Is the result as expected? Why?

In [ ]:
check_progress.test_1_3()

  <span style="font-size: 25px;">Q1. </span> __REPORT:__ Briefly explain the packet filters above ([1.1](#sec:1.1), [1.2](#sec:1.2). [1.3](#sec:1.3)). Include the packet filters in your report and also address:
   * Whether the packet filter is [stateful](https://en.wikipedia.org/wiki/Stateful_firewall) or stateless, and why?
   * Where the packet filter applies in the within the Linux networking stack, and why?


# Milestone 2 -- Securing TCP Connection: TLS

In this section, you will set up a secure TCP connection using Transport Layer Security (TLS), commonly confused with its predecessor Secure Sockets Layer (SSL), which is deprecated.
Then you will analyze a trace of TLS records to investigate the various records types and the fields in these records.

You will set up a web server that supports TLS (HTTPS: HTTP over TLS).
Then investigate how cryptography enhances HTTP with security (confidentiality, data integrity, and end-point authentication).


## Task 2.1 -- Eavesdropping Insecure Connection

* The __"webserver"__ is set up with _nginx_ but unsecured without TLS. 
Thus any established connection is susceptible to eavesdropping and tampering by an outside party.

To verify that: 

* Start packet capturing on the link connected to the __"ether0"__ interface on __"router3"__ (as the outside party, or "Trudy"). 
Retrieve the content of the webserver from "pclient2". Stop packet capturing and open the capture file in Wireshark.

* Find the HTTP response (HTTP/1.1 200 OK) sent from the __"webserver"__ to __"pclient2"__. 
Can you read the response in cleartext (line-based text data)? 

* On your **local** machine, open a web browser and navigate to your website: `http://<your_ntnu_server_IP>`. 
You can check that a web server is not secure through a browser, as in Figure 2.


|<img src="figures/figure2.png"/>|
|:--:| 
| *Figure 2: Insecure TCP connection* | 



## Task 2.2 -- Creating TLS Certificate

The first step in setting up a website with TLS support is to get a digital certificate signed by a [certification authority (CA)](https://en.wikipedia.org/wiki/Certificate_authority).
However, we can create a Self-Signed SSL Certificate using [OpenSSL](https://www.openssl.org/).


<div class="alert alert-block alert-info">
<b>Note:</b> Using OpenSSL to generate certificates doesn't mean we are using the deprecated SSL protocol.
In fact, we can generate certificates that can be used by both SSL and TLS protocols. </div>

* First, in the __"webserver"__, generate the server's private key with the following command:


<div style="background: #f8f8f8; overflow:auto;width:auto;border:solid gray;border-width:.1em .1em .1em .8em;padding:.2em .6em;"><pre style="margin: 0; line-height: 125%;color: black;">sudo openssl genrsa -out /etc/ssl/private/ttm4200-selfsigned.key 2048</pre></div>

This will generate an RSA key, 2048 bits long.
This key is required to sign the certificate.
Typically, a certificate is signed by a certificate authority (CA) using CA's private key, which will allow our browsers and other equipment to trust such keys (recall the importance of CAs on page 662 of the book).
However, we will use the private key to sign the certificate in our case.


<div class="alert alert-block alert-info">
<b>Optional:</b> If you are interested in freely generating a worldwide valid key, you may try <a href="https://letsencrypt.org" target="_blank">Let's encrypt</a>. However, take care that some of the steps may differ. </div>

* Now, you will generate the TLS certificate.
To do it, run the following command and provide the public information that will be embedded in the certificate.
Feel free to change the information __except for the Common Name__, which must correspond to the server's domain name.



   <div style="background: #f8f8f8; overflow:auto;width:auto;border:solid gray;border-width:.1em .1em .1em .8em;padding:.2em .6em;"><pre style="margin: 0; line-height: 125%;color: black;">sudo openssl req -x509 -sha256 -nodes -days 60 -key /etc/ssl/private/ttm4200-selfsigned.key -out /etc/ssl/certs/ttm4200-selfsigned.crt<br><br><span style="color: #408080; font-style: italic">#Then will need to fill out, keep the Common Name as www.ttm4200.com, and change the Email Address:</span><br>Country Name (2 letter code) [AU]:NO<br>State or Province Name (full name) [Some-State]:Trøndelag<br>Locality Name (eg, city) []:Trondheim<br>Organization Name (eg, company) [Internet Widgits Pty Ltd]:NTNU<br>Organizational Unit Name (eg, section) []:IIK<br>Common Name (e.g. server FQDN or YOUR name) []:www.ttm4200.com<br>Email Address []:aleksandra@team100.com</pre></div>

> More information about the command:
>
> __[-x509](https://en.wikipedia.org/wiki/X.509)__: define the format of public-key certificates. (<https://en.wikipedia.org/wiki/X.509>)
>
> __-sha256__: use a stronger hash algorithm
>
> __-nodes__: skip securing the certificate with a passphrase.
>
> __-days__ 60: specify the validity of the certificate
>
> __-key__: specify the private key to sign this certificate
>
> __-out__: specify the path and name of the certificate.
>
> __-Email Address__: Use an email address with your team number (e.g., aleksandra@team100.com). 
We will check this in the capture file to prove that your team generated the certificate.



* Create a Diffie–Hellman (D-H) Parameter:


   <div style="background: #f8f8f8; overflow:auto;width:auto;border:solid gray;border-width:.1em .1em .1em .8em;padding:.2em .6em;"><pre style="margin: 0; line-height: 125%;color: black;">sudo openssl dhparam -out /etc/ssl/certs/dhparam.pem 2048</pre></div>



<div class="alert alert-block alert-info">
<b>Note:</b> "Using Diffie–Hellman key exchange will generate a random and unique session key for encryption and decryption that has the additional property of forward secrecy: if the server's private key is disclosed in future, it cannot be used to decrypt the current session, even if the session is intercepted and recorded by a third party." <a href="https://en.wikipedia.org/wiki/Transport_Layer_Security" target="_blank">Reference</a>. </div>

## Task 2.3 -- Configuring the Web Server to Support TLS

The _nginx_ service installed in the __"webserver"__ is configured at "/etc/nginx/sites-available/ttm4200" to support HTTP.
You need to adjust the configuration to use TLS and redirect HTTP requests to HTTPS, thus enforcing the use of the TLS certificate. 

* Start by filling the following template. Keep in mind that HTTPS uses a different port number than HTTP.

<div class="alert alert-block alert-info">
<b>Hint:</b> Check out this <a href="http://nginx.org/en/docs/http/configuring_https_servers.html" target="_blank">link</a>. </div>


<div style="background: #f8f8f8; overflow:auto;width:auto;border:solid gray;border-width:.1em .1em .1em .8em;padding:.2em .6em;">
<pre style="margin: 0; line-height: 125%;color: black;">
server &#123;
   <span style="color: #408080; font-style: italic">#enable ssl on listening socket (IPv4, default_server)</span>
   listen ====fill in here====;<br>
   <span style="color: #408080; font-style: italic">#enable ssl on listening socket (IPv6, default_server)</span>
   listen ====fill in here====;<br>
   <span style="color: #408080; font-style: italic">#location of the server certificate</span>
   ssl_certificate ====fill in here====;<br>
   <span style="color: #408080; font-style: italic">#location of the private key</span>
   ssl_certificate_key ====fill in here====;<br>
   <span style="color: #408080; font-style: italic">#location of the dh parameter</span>
   ssl_dhparam ====fill in here====;<br>
   root /var/www/ttm4200;
   index ttm4200_index.html;
   server_name www.ttm4200.com;
   location / &#123;
      try_files $uri $uri/ =404;
   &#125;<br>
   error_page 500 502 503 504 /50x.html;
   location = /50x.html &#123;
      root /var/www/ttm4200;
      index 50x.html;
   &#125;
&#125;
</pre>
<br>

<pre style="margin: 0; line-height: 125%;color: black;">
<span style="color: #408080; font-style: italic">#catch-all http requests and redirect them to https; update redirection to your ntnu_server IP address</span> 
server &#123;
   listen 80 default_server;
   listen [::]:80 default_server;
   server_name www.ttm4200.com;
   return 301 https://www.ttm4200.com$request_uri;
&#125;
</pre>
</div>

* Check that the configuration's syntax is correct (`sudo nginx -t`) and if so, restart _nginx_ (`sudo service nginx restart`).

* On your **local** machine (Sahara's PC), open a web browser and navigate to your website: `https://<your_ntnu_server_IP>`. 
Because we are using a self-signed certificate (i.e., not signed by any trusted CA), the browser will be unable to verify the certificate presented by the webserver. 
It will issue a warning, as in Figure 3.




|<img src="figures/figure3.png" width="600" />|
|:--:| 
| *Figure 3: Self-signed certificate warning* |


* Click on _Advanced_ $\rightarrow$ _View Certificate_ to display certificate information. 
  
* __Self-Check:__ Why are the "Subject Name" and "Issuer Name" the same? 

* __Self-Check:__ What algorithm is used for public-key encryption? 

* __Self-Check:__ What is the signature algorithm? 

* __Self-Check:__ What hash function is used to produce the certificate fingerprints?

<div class="alert alert-block alert-info">
<b>Note:</b> If you already pressed <b>Accept the Risk and Continue</b> you can still view the certificate by clicking on the lock on the address bar. </div>

* In ntnu_server, try to access the webserver by retrieving the content of its HTML page:
   <div style="background: #f8f8f8; overflow:auto;width:auto;border:solid gray;border-width:.1em .1em .1em .8em;padding:.2em .6em;"><pre style="margin: 0; line-height: 125%;color: black;">wget https://127.0.0.1 --header "Host:www.ttm4200.com" --no-check-certificate<br><br>wget http://127.0.0.1 --header "Host:www.ttm4200.com" --no-check-certificate</pre></div>

> More information about the command:
>
> The request is sent to 127.0.0.1 (localhost).
>
> __--header "Host:www.ttm4200.com"__ adds a custom HTTP Host header to the request.
>
> __--no-check-certificate__ skips the SSL/TLS certificates verification.

* __Self-Check:__ Do you see the response of the previous two commands? How the system resolves `www.ttm4200.com`?

<div class="alert alert-block alert-info">
<b>Hint:</b> You might need to configure your <b>ntnu_server</b> to recognize this host. You can edit <b>/etc/hosts</b> by adding <code style="color: black;">127.0.0.1 www.ttm4200.com</code>. Then, try again the <code style="color: black;">wget</code> commands.
</div>

In [ ]:
check_progress.test_2_3()

## Task 2.4 --  Analyzing TLS protocol.


To analyze TLS, you need to capture packets in a TLS session. 

* Start a packet capture in GNS3 on the link connected to ether0 on __"router3"__. Alternatively, use tcpdump inside __"router3"__.

   <div style="background: #f8f8f8; overflow:auto;width:auto;border:solid gray;border-width:.1em .1em .1em .8em;padding:.2em .6em;"><pre style="margin: 0; line-height: 125%;color: black;">sudo tcpdump -i ether0 -w https_tls.pcap</pre></div>

* In __"pclient2"__, retrieve the content from the "webserver" using HTTPS and skip the certificate verification:

   <div style="background: #f8f8f8; overflow:auto;width:auto;border:solid gray;border-width:.1em .1em .1em .8em;padding:.2em .6em;"><pre style="margin: 0; line-height: 125%;color: black;">wget https://129.168.1.2 --header "Host: www.ttm4200.com" --no-check-certificate</pre></div>

* Stop the packet capture.
* Dump the capture to a file named "**https_tls.pcap**". Make sure to save a file with a correct file extension.
* Copy the capture file to the "~/labs/lab9" directory, then open it in Wireshark. 
* Apply the display filter `tls` to show only the frames that have TLS records. 
Then answer the following questions:


<div class="alert alert-block alert-info">
<b>Note:</b> Keep in mind that a single packet may contain one or more TLS records. </div>

* Locate the first message in the TLS handshake (_ClientHello_ record), which the client sends to initiate a session with the server. 
Then answer the following:
   * What is the packet number of _ClientHello_ record?
   * What is the value of the content type?
   * Locate the nonce (also known as a "Random"), consisting of a random number and a Unix timestamp. 
   What is the value of the nonce? How long (in bytes) is the nonce?
   * Locate the [cipher suite](https://en.wikipedia.org/wiki/Cipher_suite) list, which contains the combinations of cryptographic algorithms supported by the client in order of the client's preference (favorite choice first). 
   How many cipher suites are advertised in the Client Hello record? 
   * __Self Check:__ Locate the "supported_versions" Extension. 
   Why is the client requesting a TLS 1.2 handshake when it can support TLS 1.3? [HINT](https://www.thesecmaster.com/decoding-tls-1-3-protocol-handshake-with-wireshark/)

In [ ]:
# What is the packet number of _ClientHello_ record?
ClientHello_packet_number = # your answer (as integer)

# What is the value of the content type?
ClientHello_content_type = # your answer (as integer)

# Locate the nonce (also known as a "Random"), consisting of a random number and a Unix timestamp. What is the value of the nonce? 
ClientHello_nonce_value = # your answer (as a string of hexadecimal value)
# How long (in bytes) is the nonce? 
ClientHello_nonce_length = # your answer (as integer) 

# How many cipher suites are advertised in the Client Hello record? 
Number_of_cipher_suites = # your answer (as integer)


check_progress.test_2_4_1(ClientHello_packet_number, ClientHello_content_type, ClientHello_nonce_value , ClientHello_nonce_length, Number_of_cipher_suites)

* Locate the _ServerHello_ record and expand it. 
Then answer the following:
    * What is the packet number containing the _ServerHello_ record?
    * Which TLS version does the server support?
    * Find the chosen cipher suite (the server-chosen cipher suite from the client's advertised list) and specify the selected algorithms:
      * The symmetric-key algorithm (bulk encryption algorithm) is used to encrypt the sent data.
      * The hash algorithm (message authentication code algorithm)
 

* __Self Check:__ Can you locate the _Certificate_, _Server Key Exchange_, and _Client Key Exchange_ records? Why?

In [ ]:
# What is the packet number containing the _ServerHello_ record?
ServerHello_packet_number =  # your answer (as integer)

# Which TLS version does the server support?
Server_suppored_TLS_version =  # your answer (either 1.0, 1.1, 1.2, 1.3)

# Find the chosen cipher suite in the ServerHello_and specify the selected algorithms:
symmetric_key_algorithm = # your answer (as string)
hash_algorithm = # your answer (as string)


check_progress.test_2_4_2(ServerHello_packet_number, Server_suppored_TLS_version, symmetric_key_algorithm, hash_algorithm)

  <span style="font-size: 25px;">Q2. </span> __REPORT:__ Draw a sequence diagram of all TLS __records__ exchanged during TLS handshake. Then:
   * Explain the functionality of each TLS record.
   * Explain how the application data is encrypted.

<div class="alert alert-block alert-info">
<b>Note:</b> Keep in mind that a single packet may contain one or more records. 
The question is about TLS <b>records</b>, not packets or frames. </div>
